In [ ]:
import sys, pathlib
root = pathlib.Path().resolve().parents[0]
sys.path.insert(0, str(root))
from BESTLIB import MatrixLayout

# HTML simple
MatrixLayout.set_safe_html(True)
MatrixLayout.map({
    "A": "<b style='color:blue'>Title</b>",
    "C": "<i>Control</i>",
    "B": "<b style='color:red'>ROJO<b>"
})

layout = MatrixLayout("""
AAC
AAC
BBB
""")

layout  # Render HTML